# 9. Consolidate observations

## Setup

In [1]:
library(tidyverse)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
sample_file <- '../preprocessed_data/sample_2019-09-02.feather'
company_groups_file <- '../preprocessed_data/company_groups_2019-09-02.feather'
m_as_file <- '../preprocessed_data/m_as_2019-09-02.feather'
pipelines_2004_file <- "../preprocessed_data/pipelines_2004_selected_2019-08-26.feather"

## Read data

In [3]:
sample <- feather::read_feather(sample_file)
testit::assert(nrow(sample[duplicated(select(sample, YEAR, COMMODITY, OPERATOR_ID)), ]) == 0)
glimpse(sample)

Warning message:
“Coercing int64 to double”

Observations: 1,756
Variables: 20
$ OPERATOR_ID           <chr> "300", "300", "300", "300", "300", "300", "300"…
$ YEAR                  <dbl> 2017, 2017, 2011, 2014, 2012, 2010, 2018, 2018,…
$ NAME                  <chr> "PLAINS PIPELINE, L.P.", "PLAINS PIPELINE, L.P.…
$ COMMODITY             <chr> "hvl", "non-hvl", "non-hvl", "non-hvl", "non-hv…
$ MILES                 <dbl> 72.90, 0.00, 326.80, NA, 265.17, 62.28, 29.05, …
$ AGE_UNKNOWN_MILES     <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_PRE_1940        <dbl> 0.00, 3.72, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_1940            <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 15, 15,…
$ MILES_1950            <dbl> 0.00, 0.00, 0.80, 0.00, 0.80, 244.98, 0.00, 0.0…
$ MILES_1960            <dbl> 0.00, 0.00, 188.43, 0.00, 188.43, 4.76, 0.00, 0…
$ MILES_1970            <dbl> 0.00, 0.00, 1.17, 0.00, 1.17, 0.00, 0.00, 0.00,…
$ MILES_1980            <dbl> 0.00, 0.00, 4.43, 0.00, 4.43, 0.00, 0.00, 0.00,…
$ MILES_1990      

In [4]:
company_groups <- feather::read_feather(company_groups_file)
glimpse(company_groups)

Observations: 97
Variables: 2
$ members <chr> "31618", "3445", "30829", "31270", "22610", "12105", "31579",…
$ name    <chr> "Enterprise Products (Group)", "Enterprise Products (Group)",…


In [5]:
m_as_orig <- feather::read_feather(m_as_file)
glimpse(m_as_orig)

Observations: 13
Variables: 4
$ members    <chr> "Sunoco (Group)", "32683", "22442", "Sunoco (Group)", "320…
$ name       <chr> "Sunoco (Group)", "Sunoco (Group)", "Sunoco (Group)", "Sun…
$ start_year <chr> "2011", "2011", "2011", "2013", "2013", "2017", "2017", NA…
$ end_year   <chr> NA, NA, NA, NA, NA, NA, NA, "2011", "2011", NA, NA, "2017"…


In [6]:
test <- select(sample, COMMODITY, YEAR, OPERATOR_ID)

In [7]:
pipelines_2004 <- feather::read_feather(pipelines_2004_file)
glimpse(pipelines_2004)

Warning message:
“Coercing int64 to double”

Observations: 2,730
Variables: 16
$ OPERATOR_ID    <chr> "31336", "4805", "8175", "26302", "32147", "4906", "19…
$ YEAR           <dbl> 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008, …
$ NAME           <chr> "CHEVRON U.S.A. INC", "EXPLORER PIPELINE CO", "INTERST…
$ COMMODITY      <chr> "CRUDE OIL", "PETROLEUM & REFINED PRODUCTS", "PETROLEU…
$ MILES          <dbl> 6.000, 1204.110, 8.000, 13.000, 1629.158, 373.000, 40.…
$ MILES_PRE_1940 <dbl> 0.000, 0.000, 0.000, 0.000, 26.841, 1.000, 0.000, 0.00…
$ MILES_1940     <dbl> 0.000, 0.000, 0.000, 0.000, 119.533, 101.000, 0.000, 0…
$ MILES_1950     <dbl> 0.000, 7.200, 0.000, 0.000, 585.263, 126.000, 0.000, 0…
$ MILES_1960     <dbl> 10.647, 7.800, 11.600, 0.000, 107.873, 342.000, 0.000,…
$ MILES_1970     <dbl> 0.000, 593.200, 0.000, 0.000, 243.374, 43.000, 0.000, …
$ MILES_1980     <dbl> 0.000, 18.300, 0.000, 89.000, 91.639, 45.000, 0.000, 0…
$ MILES_1990     <dbl> 0.000, 20.800, 0.000, 0.000, 110.908, 42.000, 0.000, 0…
$ MILES_2000     <

## 9.1 Add two years of data for crude, non-HVL and HVL.

Adding the data for crude the years 2008/2009 offers us a relatively easy way to expand our sample. We cannot go any further, as definition of incidents was change, 2008/2009 can still be used to calculate the three years average that we use to calculate the impact of change on the incidents rates for the observation 2010-2012.

In [8]:
pipelines_2004$COMMODITY <- fct_recode(pipelines_2004$COMMODITY, 
                                       "crude" = "CRUDE OIL", "hvl" = "HVLS", "non-hvl" = "PETROLEUM & REFINED PRODUCTS")
table(pipelines_2004$COMMODITY)


CO2 OR OTHER        crude          hvl      non-hvl 
         168          823          863          876 

In [9]:
commodity_operator_pairs <- unite(sample, OPERATOR_ID, COMMODITY, col="pairs", sep="")$pairs
pipelines_2008 <- pipelines_2004 %>%
    filter(YEAR >= 2008) %>%
    unite(OPERATOR_ID, COMMODITY, col="pairs", sep="", remove=FALSE) %>%
    filter(pairs %in% commodity_operator_pairs) %>%
    select(-c("pairs"))

glimpse(pipelines_2008)

Observations: 336
Variables: 16
$ OPERATOR_ID    <chr> "4805", "32147", "4906", "26041", "31618", "30755", "3…
$ YEAR           <dbl> 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008, …
$ NAME           <chr> "EXPLORER PIPELINE CO", "MARATHON PIPE LINE LLC", "EXX…
$ COMMODITY      <fct> non-hvl, non-hvl, non-hvl, non-hvl, hvl, hvl, hvl, hvl…
$ MILES          <dbl> 1204.110, 1629.158, 373.000, 2.000, 3346.770, 109.895,…
$ MILES_PRE_1940 <dbl> 0.000, 26.841, 1.000, 0.000, 24.131, 0.000, 0.000, 0.0…
$ MILES_1940     <dbl> 0.000, 119.533, 101.000, 0.000, 343.831, 0.000, 64.150…
$ MILES_1950     <dbl> 7.200, 585.263, 126.000, 0.000, 31.950, 0.000, 33.900,…
$ MILES_1960     <dbl> 7.800, 107.873, 342.000, 0.000, 3267.673, 0.000, 0.000…
$ MILES_1970     <dbl> 593.200, 243.374, 43.000, 0.000, 432.330, 40.515, 7.31…
$ MILES_1980     <dbl> 18.300, 91.639, 45.000, 2.000, 411.570, 0.000, 0.000, …
$ MILES_1990     <dbl> 20.800, 110.908, 42.000, 0.000, 518.873, 69.380, 0.000…
$ MILES_2000     <db

In [10]:
pipelines_2008 %>%
    mutate(PERC_OFFSHORE = round(PERC_OFFSHORE, 2)) %>%
    {table(.$PERC_OFFSHORE)}


  0 
336 

### 9.1.1 Consolidate observations

Before 2010, business units were sometimes reported separately, within the same OPERATOR ID. We consolidate those at the OPERATOR ID level.

In [11]:
pipelines_2008 <- pipelines_2008 %>%
    group_by(OPERATOR_ID, YEAR, COMMODITY) %>%
    summarize(NAME = first(NAME),
              MILES = sum(MILES), 
              MILES_PRE_1940 = sum(MILES_PRE_1940), 
              MILES_1940 = sum(MILES_1940), 
              MILES_1950 = sum(MILES_1950), 
              MILES_1960 = sum(MILES_1960), 
              MILES_1970 = sum(MILES_1970), 
              MILES_1980 = sum(MILES_1980), 
              MILES_1990 = sum(MILES_1990), 
              MILES_2000 = sum(MILES_2000), 
              MILES_2010 = sum(MILES_2010)) %>%
    ungroup()

In [12]:
pipelines_2008$AVG_AGE = (pipelines_2008$MILES_PRE_1940 * 90 +
                          pipelines_2008$MILES_1940 * 75 +
                          pipelines_2008$MILES_1950 * 65 +
                          pipelines_2008$MILES_1960 * 55 +
                          pipelines_2008$MILES_1970 * 45 +
                          pipelines_2008$MILES_1980 * 35 +
                          pipelines_2008$MILES_1990 * 25 +
                          pipelines_2008$MILES_2000 * 15 +
                          pipelines_2008$MILES_2010 * 5) / 
                         (pipelines_2008$MILES_PRE_1940 + 
                          pipelines_2008$MILES_1940 + 
                          pipelines_2008$MILES_1950 + 
                          pipelines_2008$MILES_1960 + 
                          pipelines_2008$MILES_1970 + 
                          pipelines_2008$MILES_1980 + 
                          pipelines_2008$MILES_1990 + 
                          pipelines_2008$MILES_2000 + 
                          pipelines_2008$MILES_2010)

### 9.1.2 Merge

In [13]:
colnames(sample) [!(colnames(sample) %in% colnames(pipelines_2008))]

[1] "AGE_UNKNOWN_MILES"     "PERC_OFFSHORE"         "PARENT"               
[4] "INCIDENTS"             "SIGNIFICANT_INCIDENTS"

In [14]:
pipelines_2008$AGE_UNKNOWN_MILES <- NA
pipelines_2008$PERC_OFFSHORE <- NA
pipelines_2008$PARENT <- NA
pipelines_2008$INCIDENTS <- NA
pipelines_2008$SIGNIFICANT_INCIDENTS <- NA

In [15]:
testit::assert(nrow(subset(pipelines_2008, YEAR == 2008)) > 0)
sample <- rbind(sample, pipelines_2008)

## 9.2 Calculate change

In [16]:
sample <- sample %>%
    group_by(OPERATOR_ID, COMMODITY) %>%
    arrange(YEAR) %>%
    mutate(CHANGE = MILES - lag(MILES, 1))
glimpse(subset(sample, YEAR > 2010))

Observations: 1,580
Variables: 21
Groups: OPERATOR_ID, COMMODITY [242]
$ OPERATOR_ID           <chr> "300", "395", "473", "473", "473", "994", "1248…
$ YEAR                  <dbl> 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011,…
$ NAME                  <chr> "PLAINS PIPELINE, L.P.", "AMOCO OIL CO", "ANADA…
$ COMMODITY             <chr> "non-hvl", "non-hvl", "co2", "crude", "hvl", "h…
$ MILES                 <dbl> 326.800, 527.000, NA, 12.000, NA, 121.400, 126.…
$ AGE_UNKNOWN_MILES     <dbl> 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000…
$ MILES_PRE_1940        <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ MILES_1940            <dbl> 0.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8…
$ MILES_1950            <dbl> 0.800, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000…
$ MILES_1960            <dbl> 188.430, 1.000, 0.000, 43.000, 0.000, 121.000, …
$ MILES_1970            <dbl> 1.170, 2.000, 0.000, 16.000, 0.000, 1.000, 189.…
$ MILES_1980            <dbl> 4.430, 17.000, 0.000, 0.000, 2

## 9.3 Consolidate observations

### 9.3.1 Company groups

In [17]:
company_groups <- rename(company_groups, new_name = name)
sample <- left_join(sample, company_groups, by=c('OPERATOR_ID' = 'members'))
testit::assert(nrow(sample[duplicated(select(sample, YEAR, COMMODITY, OPERATOR_ID)), ]) == 0)
glimpse(subset(sample, !is.na(sample$new_name)))

Observations: 1,356
Variables: 22
Groups: OPERATOR_ID, COMMODITY [162]
$ OPERATOR_ID           <chr> "10012", "10012", "11169", "11169", "11551", "1…
$ YEAR                  <dbl> 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008,…
$ NAME                  <chr> "NUSTAR PIPELINE OPERATING PARTNERSHIP L.P.", "…
$ COMMODITY             <chr> "hvl", "non-hvl", "crude", "hvl", "crude", "non…
$ MILES                 <dbl> 1441.630, 477.590, 1030.000, 118.000, 85.300, 1…
$ AGE_UNKNOWN_MILES     <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,…
$ MILES_PRE_1940        <dbl> 0.0, 0.0, 0.0, 0.0, 219.0, 11.0, 0.0, 0.0, 0.0,…
$ MILES_1940            <dbl> 0.000, 0.578, 0.000, 0.000, 0.000, 0.000, 0.000…
$ MILES_1950            <dbl> 131.445, 29.452, 0.000, 0.000, 21.000, 0.000, 0…
$ MILES_1960            <dbl> 0.000, 399.006, 2.000, 0.000, 0.000, 0.000, 673…
$ MILES_1970            <dbl> 0.000, 36.655, 4.000, 0.000, 3.000, 8.000, 404.…
$ MILES_1980            <dbl> 0.831, 128.756, 1.000, 0.000, 

In [18]:
nrow(sample[is.na(sample$MILES), ])

[1] 40

In [19]:
sample_no_group <- subset(sample, is.na(sample$new_name))
sample_group <- subset(sample, !is.na(sample$new_name))

sample_group <- sample_group %>%
    group_by(new_name, YEAR, COMMODITY) %>%
    summarize(OPERATOR_ID = first(new_name), 
              NAME              = first(new_name),
              MILES             = sum(MILES, na.rm=TRUE), 
              AGE_UNKNOWN_MILES = sum(AGE_UNKNOWN_MILES, na.rm=TRUE), 
              MILES_PRE_1940    = sum(MILES_PRE_1940, na.rm=TRUE), 
              MILES_1940      = sum(MILES_1940, na.rm=TRUE),
              MILES_1950      = sum(MILES_1950, na.rm=TRUE),
              MILES_1960      = sum(MILES_1960, na.rm=TRUE),
              MILES_1970      = sum(MILES_1970, na.rm=TRUE),
              MILES_1980      = sum(MILES_1980, na.rm=TRUE),
              MILES_1990      = sum(MILES_1990, na.rm=TRUE),
              MILES_2000      = sum(MILES_2000, na.rm=TRUE),
              MILES_2010      = sum(MILES_2010, na.rm=TRUE),
              PARENT            = first(PARENT), 
              INCIDENTS         = sum(INCIDENTS, na.rm=TRUE), 
              SIGNIFICANT_INCIDENTS = sum(SIGNIFICANT_INCIDENTS, na.rm=TRUE), 
              GROUP = 'group') %>%
    # We calculate the avg age and change once all groupings (including M&As below), are taken care of
    ungroup()

glimpse(sample_group)

Observations: 739
Variables: 20
$ new_name              <chr> "BKEP (Group)", "BKEP (Group)", "BKEP (Group)",…
$ YEAR                  <dbl> 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,…
$ COMMODITY             <chr> "crude", "crude", "crude", "crude", "crude", "c…
$ OPERATOR_ID           <chr> "BKEP (Group)", "BKEP (Group)", "BKEP (Group)",…
$ NAME                  <chr> "BKEP (Group)", "BKEP (Group)", "BKEP (Group)",…
$ MILES                 <dbl> 90.00, 141.54, 141.54, 196.27, 217.20, 270.10, …
$ AGE_UNKNOWN_MILES     <dbl> 0.00, 39.23, 39.23, 39.28, 39.70, 0.00, 67.50, …
$ MILES_PRE_1940        <dbl> 8.00, 0.00, 0.00, 0.00, 0.00, 27.40, 27.40, 27.…
$ MILES_1940            <dbl> 5.00, 6.98, 6.98, 6.97, 55.20, 55.20, 55.20, 56…
$ MILES_1950            <dbl> 1.000, 3.140, 3.140, 3.140, 3.100, 3.200, 3.200…
$ MILES_1960            <dbl> 2.00, 2.81, 2.81, 2.81, 0.80, 0.80, 0.80, 1.30,…
$ MILES_1970            <dbl> 6.00, 11.36, 11.36, 11.68, 9.10, 10.60, 10.60, …
$ MILES_1980        

In [20]:
sample_no_group$GROUP = "not group"

sample <- bind_rows(sample_group, sample_no_group)
sample <- sample[ , (names(sample) != "new_name")]
testit::assert(nrow(sample[duplicated(select(sample, YEAR, COMMODITY, OPERATOR_ID)), ]) == 0)
glimpse(sample_n(sample, 10))

Observations: 10
Variables: 22
$ YEAR                  <dbl> 2018, 2012, 2013, 2008, 2017, 2012, 2012, 2010,…
$ COMMODITY             <chr> "co2", "crude", "crude", "crude", "hvl", "non-h…
$ OPERATOR_ID           <chr> "Occidental Petroleum (Group)", "Phillips 66 (G…
$ NAME                  <chr> "Occidental Petroleum (Group)", "Phillips 66 (G…
$ MILES                 <dbl> 217.50, 1120.93, 1025.40, 145.00, 32.74, 271.44…
$ AGE_UNKNOWN_MILES     <dbl> 0.00, 454.01, 126.00, 0.00, 0.00, 0.00, 0.00, 9…
$ MILES_PRE_1940        <dbl> 0.0, 0.0, 0.0, 54.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0…
$ MILES_1940            <dbl> 0.00, 518.92, 440.00, 10.90, 0.00, 0.00, 0.00, …
$ MILES_1950            <dbl> 0.00, 745.66, 209.00, 0.00, 0.00, 0.80, 84.56, …
$ MILES_1960            <dbl> 0.00, 627.97, 36.20, 0.00, 0.00, 188.43, 46.37,…
$ MILES_1970            <dbl> 0.00, 57.70, 127.10, 0.00, 0.00, 4.95, 0.00, 0.…
$ MILES_1980            <dbl> 646.83, 40.66, 82.00, 18.90, 0.00, 4.43, 32.58,…
$ MILES_1990         

In [21]:
testit::assert(nrow(subset(sample, YEAR == 2008)) > 0)
table(sample$GROUP)


    group not group 
      739       715 

### 9.3.2 M&As

In [22]:
m_as <- rename(m_as_orig, new_name = name)

# We need to fill in the NA values for start and end_year with values that always match, because when logical conditions encounter NAs
# they will always be wrong. The filtering in those cases happens because the new_name column is NA.
m_as[is.na(m_as$start_year), ]$start_year <- -9999
m_as[is.na(m_as$end_year), ]$end_year <- 9999

years = data.frame(year = unique(sample$YEAR))
m_as <- merge(m_as, years)
m_as <- m_as %>%
    filter(year >= start_year & year < end_year) %>%
    select(members, new_name, year) %>%
    distinct()
m_as$m_a = TRUE

glimpse(m_as)

Observations: 60
Variables: 4
$ members  <chr> "32288", "31476", "SemGroup (Group)", "Glass Mountain (Group…
$ new_name <chr> "SemGroup (Group)", "SemGroup (Group)", "SemGroup (Group)", …
$ year     <dbl> 2009, 2009, 2009, 2009, 2010, 2010, 2010, 2010, 2011, 2011, …
$ m_a      <lgl> TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, …


In [23]:
sample <- left_join(sample, m_as, by=c('OPERATOR_ID' = 'members', 'YEAR' = 'year'))
sample$m_a <- replace_na(sample$m_a, FALSE)

glimpse(sample)

Observations: 1,454
Variables: 24
$ YEAR                  <dbl> 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,…
$ COMMODITY             <chr> "crude", "crude", "crude", "crude", "crude", "c…
$ OPERATOR_ID           <chr> "BKEP (Group)", "BKEP (Group)", "BKEP (Group)",…
$ NAME                  <chr> "BKEP (Group)", "BKEP (Group)", "BKEP (Group)",…
$ MILES                 <dbl> 90.00, 141.54, 141.54, 196.27, 217.20, 270.10, …
$ AGE_UNKNOWN_MILES     <dbl> 0.00, 39.23, 39.23, 39.28, 39.70, 0.00, 67.50, …
$ MILES_PRE_1940        <dbl> 8.00, 0.00, 0.00, 0.00, 0.00, 27.40, 27.40, 27.…
$ MILES_1940            <dbl> 5.00, 6.98, 6.98, 6.97, 55.20, 55.20, 55.20, 56…
$ MILES_1950            <dbl> 1.000, 3.140, 3.140, 3.140, 3.100, 3.200, 3.200…
$ MILES_1960            <dbl> 2.00, 2.81, 2.81, 2.81, 0.80, 0.80, 0.80, 1.30,…
$ MILES_1970            <dbl> 6.00, 11.36, 11.36, 11.68, 9.10, 10.60, 10.60, …
$ MILES_1980            <dbl> 1.00, 0.55, 0.55, 0.58, 0.60, 0.60, 0.60, 0.70,…
$ MILES_1990      

In [24]:
sample_m_as <- subset(sample, m_a == TRUE)
sample_no_m_as <- subset(sample, m_a == FALSE)

glimpse(sample_m_as)

Observations: 65
Variables: 24
$ YEAR                  <dbl> 2014, 2015, 2016, 2011, 2011, 2011, 2012, 2012,…
$ COMMODITY             <chr> "crude", "crude", "crude", "crude", "hvl", "non…
$ OPERATOR_ID           <chr> "Glass Mountain (Group)", "Glass Mountain (Grou…
$ NAME                  <chr> "Glass Mountain (Group)", "Glass Mountain (Grou…
$ MILES                 <dbl> 69.30, 72.50, 98.83, 2161.74, 171.39, 1588.06, …
$ AGE_UNKNOWN_MILES     <dbl> 0.00, 0.00, 0.00, 371.80, 0.00, 0.00, 371.80, 0…
$ MILES_PRE_1940        <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_1940            <dbl> 0.00, 0.00, 0.00, 10.00, 0.00, 151.84, 10.00, 0…
$ MILES_1950            <dbl> 0.00, 0.00, 0.00, 276.30, 0.00, 16.94, 276.30, …
$ MILES_1960            <dbl> 0.00, 0.00, 0.00, 183.68, 11.00, 206.42, 183.68…
$ MILES_1970            <dbl> 0.00, 0.00, 0.00, 69.00, 6.00, 0.00, 69.00, 6.0…
$ MILES_1980            <dbl> 0.00, 0.00, 0.00, 23.00, 5.00, 41.00, 23.00, 5.…
$ MILES_1990         

In [25]:
sample_m_as <- sample_m_as %>%
    group_by(new_name, YEAR, COMMODITY) %>%
    summarize(OPERATOR_ID = first(new_name), 
              NAME              = first(new_name),
              MILES             = sum(MILES, na.rm=TRUE), 
              AGE_UNKNOWN_MILES = sum(AGE_UNKNOWN_MILES, na.rm=TRUE), 
              MILES_PRE_1940    = sum(MILES_PRE_1940, na.rm=TRUE), 
              MILES_1940        = sum(MILES_1940, na.rm=TRUE),
              MILES_1950        = sum(MILES_1950, na.rm=TRUE),
              MILES_1960        = sum(MILES_1960, na.rm=TRUE),
              MILES_1970        = sum(MILES_1970, na.rm=TRUE),
              MILES_1980        = sum(MILES_1980, na.rm=TRUE),
              MILES_1990        = sum(MILES_1990, na.rm=TRUE),
              MILES_2000        = sum(MILES_2000, na.rm=TRUE),
              MILES_2010        = sum(MILES_2010, na.rm=TRUE),
              PARENT            = first(PARENT), 
              INCIDENTS         = sum(INCIDENTS, na.rm=TRUE), 
              SIGNIFICANT_INCIDENTS = sum(SIGNIFICANT_INCIDENTS, na.rm=TRUE), 
              GROUP = 'm&a') %>%
    ungroup()

glimpse(sample_m_as)

Observations: 36
Variables: 20
$ new_name              <chr> "SemGroup (Group)", "SemGroup (Group)", "SemGro…
$ YEAR                  <dbl> 2008, 2009, 2010, 2014, 2015, 2016, 2011, 2011,…
$ COMMODITY             <chr> "crude", "crude", "crude", "crude", "crude", "c…
$ OPERATOR_ID           <chr> "SemGroup (Group)", "SemGroup (Group)", "SemGro…
$ NAME                  <chr> "SemGroup (Group)", "SemGroup (Group)", "SemGro…
$ MILES                 <dbl> 119.00, 138.00, 291.25, 69.30, 72.50, 98.83, 23…
$ AGE_UNKNOWN_MILES     <dbl> 0.00, 0.00, 93.19, 0.00, 0.00, 0.00, 371.80, 0.…
$ MILES_PRE_1940        <dbl> 22.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, …
$ MILES_1940            <dbl> 22.00, 0.00, 31.87, 0.00, 0.00, 0.00, 10.00, 0.…
$ MILES_1950            <dbl> 146.00, 146.00, 220.67, 0.00, 0.00, 0.00, 276.3…
$ MILES_1960            <dbl> 10.00, 9.00, 67.82, 0.00, 0.00, 0.00, 183.68, 1…
$ MILES_1970            <dbl> 6.00, 0.00, 0.00, 0.00, 0.00, 0.00, 69.00, 6.00…
$ MILES_1980         

In [26]:
sample <- bind_rows(sample_m_as, sample_no_m_as)
sample <- sample[ , !(names(sample) %in% c("new_name", "m_a"))]
testit::assert(nrow(sample[duplicated(select(sample, YEAR, COMMODITY, OPERATOR_ID)), ]) == 0)
testit::assert(nrow(subset(sample, YEAR == 2008)) > 0)
glimpse(sample)

Observations: 1,425
Variables: 22
$ YEAR                  <dbl> 2008, 2009, 2010, 2014, 2015, 2016, 2011, 2011,…
$ COMMODITY             <chr> "crude", "crude", "crude", "crude", "crude", "c…
$ OPERATOR_ID           <chr> "SemGroup (Group)", "SemGroup (Group)", "SemGro…
$ NAME                  <chr> "SemGroup (Group)", "SemGroup (Group)", "SemGro…
$ MILES                 <dbl> 119.00, 138.00, 291.25, 69.30, 72.50, 98.83, 23…
$ AGE_UNKNOWN_MILES     <dbl> 0.00, 0.00, 93.19, 0.00, 0.00, 0.00, 371.80, 0.…
$ MILES_PRE_1940        <dbl> 22.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, …
$ MILES_1940            <dbl> 22.00, 0.00, 31.87, 0.00, 0.00, 0.00, 10.00, 0.…
$ MILES_1950            <dbl> 146.00, 146.00, 220.67, 0.00, 0.00, 0.00, 276.3…
$ MILES_1960            <dbl> 10.00, 9.00, 67.82, 0.00, 0.00, 0.00, 183.68, 1…
$ MILES_1970            <dbl> 6.00, 0.00, 0.00, 0.00, 0.00, 0.00, 69.00, 6.00…
$ MILES_1980            <dbl> 2.00, 1.00, 0.58, 0.00, 0.00, 0.00, 23.00, 5.00…
$ MILES_1990      

## 9.4 Mark M&A years

In [27]:
glimpse(m_as_orig)

Observations: 13
Variables: 4
$ members    <chr> "Sunoco (Group)", "32683", "22442", "Sunoco (Group)", "320…
$ name       <chr> "Sunoco (Group)", "Sunoco (Group)", "Sunoco (Group)", "Sun…
$ start_year <chr> "2011", "2011", "2011", "2013", "2013", "2017", "2017", NA…
$ end_year   <chr> NA, NA, NA, NA, NA, NA, NA, "2011", "2011", NA, NA, "2017"…


In [28]:
m_as_orig$start_year <- as.numeric(m_as_orig$start_year)
m_as_orig <- select(m_as_orig, -c(name))

m_as_start <- subset(m_as_orig, !is.na(start_year))
m_as_start$m_a_start <- TRUE

sample <- left_join(sample, m_as_start, by=c("OPERATOR_ID" = "members", "YEAR" = "start_year"))
testit::assert(nrow(subset(sample, YEAR == 2008)) > 0)
glimpse(sample)

Observations: 1,425
Variables: 24
$ YEAR                  <dbl> 2008, 2009, 2010, 2014, 2015, 2016, 2011, 2011,…
$ COMMODITY             <chr> "crude", "crude", "crude", "crude", "crude", "c…
$ OPERATOR_ID           <chr> "SemGroup (Group)", "SemGroup (Group)", "SemGro…
$ NAME                  <chr> "SemGroup (Group)", "SemGroup (Group)", "SemGro…
$ MILES                 <dbl> 119.00, 138.00, 291.25, 69.30, 72.50, 98.83, 23…
$ AGE_UNKNOWN_MILES     <dbl> 0.00, 0.00, 93.19, 0.00, 0.00, 0.00, 371.80, 0.…
$ MILES_PRE_1940        <dbl> 22.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, …
$ MILES_1940            <dbl> 22.00, 0.00, 31.87, 0.00, 0.00, 0.00, 10.00, 0.…
$ MILES_1950            <dbl> 146.00, 146.00, 220.67, 0.00, 0.00, 0.00, 276.3…
$ MILES_1960            <dbl> 10.00, 9.00, 67.82, 0.00, 0.00, 0.00, 183.68, 1…
$ MILES_1970            <dbl> 6.00, 0.00, 0.00, 0.00, 0.00, 0.00, 69.00, 6.00…
$ MILES_1980            <dbl> 2.00, 1.00, 0.58, 0.00, 0.00, 0.00, 23.00, 5.00…
$ MILES_1990      

In [29]:
m_as_orig$end_year <- as.numeric(m_as_orig$end_year)

m_as_end <- subset(m_as_orig, !is.na(end_year))
# We add 1 to the year, so the year we match on is the first year after the group has ceased to exist.
m_as_end$end_year <- m_as_end$end_year + 1
m_as_end$m_a_end <- TRUE

sample <- left_join(sample, m_as_end, by=c("OPERATOR_ID" = "members", "YEAR" = "end_year"))
testit::assert(nrow(subset(sample, YEAR == 2008)) > 0)
glimpse(sample)

Observations: 1,425
Variables: 26
$ YEAR                  <dbl> 2008, 2009, 2010, 2014, 2015, 2016, 2011, 2011,…
$ COMMODITY             <chr> "crude", "crude", "crude", "crude", "crude", "c…
$ OPERATOR_ID           <chr> "SemGroup (Group)", "SemGroup (Group)", "SemGro…
$ NAME                  <chr> "SemGroup (Group)", "SemGroup (Group)", "SemGro…
$ MILES                 <dbl> 119.00, 138.00, 291.25, 69.30, 72.50, 98.83, 23…
$ AGE_UNKNOWN_MILES     <dbl> 0.00, 0.00, 93.19, 0.00, 0.00, 0.00, 371.80, 0.…
$ MILES_PRE_1940        <dbl> 22.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, …
$ MILES_1940            <dbl> 22.00, 0.00, 31.87, 0.00, 0.00, 0.00, 10.00, 0.…
$ MILES_1950            <dbl> 146.00, 146.00, 220.67, 0.00, 0.00, 0.00, 276.3…
$ MILES_1960            <dbl> 10.00, 9.00, 67.82, 0.00, 0.00, 0.00, 183.68, 1…
$ MILES_1970            <dbl> 6.00, 0.00, 0.00, 0.00, 0.00, 0.00, 69.00, 6.00…
$ MILES_1980            <dbl> 2.00, 1.00, 0.58, 0.00, 0.00, 0.00, 23.00, 5.00…
$ MILES_1990      

In [30]:
subset(sample, m_a_end == TRUE)

YEAR,COMMODITY,OPERATOR_ID,NAME,MILES,AGE_UNKNOWN_MILES,MILES_PRE_1940,MILES_1940,MILES_1950,MILES_1960,⋯,INCIDENTS,SIGNIFICANT_INCIDENTS,GROUP,PERC_OFFSHORE,AVG_AGE,CHANGE,end_year,m_a_start,start_year,m_a_end
<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>,<dbl>,<lgl>
2018,crude,Glass Mountain (Group),Glass Mountain (Group),79.35,0.00,0,0.00,0.00,0.00,⋯,0,0,group,NA,NA,NA,NA,NA,NA,TRUE
2012,crude,31476,ROSE ROCK MIDSTREAM L.P.,168.44,60.14,0,24.15,164.75,66.91,⋯,3,3,not group,0,60.58908,2.71,NA,NA,NA,TRUE
2012,crude,32288,"WHITE CLIFFS PIPELINE, LLC",122.70,0.00,0,0.00,0.00,0.00,⋯,0,0,not group,0,15.00000,13.70,NA,NA,NA,TRUE


In [31]:
sample$M_A <- (sample$m_a_start | sample$m_a_end)
sample <- select(sample, -c(end_year, m_a_start, start_year, m_a_end))
sample$M_A <- replace_na(sample$M_A, FALSE)

glimpse(sample)

Observations: 1,425
Variables: 23
$ YEAR                  <dbl> 2008, 2009, 2010, 2014, 2015, 2016, 2011, 2011,…
$ COMMODITY             <chr> "crude", "crude", "crude", "crude", "crude", "c…
$ OPERATOR_ID           <chr> "SemGroup (Group)", "SemGroup (Group)", "SemGro…
$ NAME                  <chr> "SemGroup (Group)", "SemGroup (Group)", "SemGro…
$ MILES                 <dbl> 119.00, 138.00, 291.25, 69.30, 72.50, 98.83, 23…
$ AGE_UNKNOWN_MILES     <dbl> 0.00, 0.00, 93.19, 0.00, 0.00, 0.00, 371.80, 0.…
$ MILES_PRE_1940        <dbl> 22.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, …
$ MILES_1940            <dbl> 22.00, 0.00, 31.87, 0.00, 0.00, 0.00, 10.00, 0.…
$ MILES_1950            <dbl> 146.00, 146.00, 220.67, 0.00, 0.00, 0.00, 276.3…
$ MILES_1960            <dbl> 10.00, 9.00, 67.82, 0.00, 0.00, 0.00, 183.68, 1…
$ MILES_1970            <dbl> 6.00, 0.00, 0.00, 0.00, 0.00, 0.00, 69.00, 6.00…
$ MILES_1980            <dbl> 2.00, 1.00, 0.58, 0.00, 0.00, 0.00, 23.00, 5.00…
$ MILES_1990      

## 9.5 Calculate avg age, change for new groups

In [32]:
testit::assert(nrow(subset(sample, YEAR == 2008)) > 0)
sample <- sample %>%
    group_by(OPERATOR_ID, COMMODITY) %>%
    arrange(YEAR) %>%
    mutate(CHANGE = MILES - lag(MILES, 1))
glimpse(subset(sample, YEAR > 2010))

Observations: 1,073
Variables: 23
Groups: OPERATOR_ID, COMMODITY [160]
$ YEAR                  <dbl> 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011,…
$ COMMODITY             <chr> "crude", "hvl", "non-hvl", "crude", "crude", "h…
$ OPERATOR_ID           <chr> "Sunoco (Group)", "Sunoco (Group)", "Sunoco (Gr…
$ NAME                  <chr> "Sunoco (Group)", "Sunoco (Group)", "Sunoco (Gr…
$ MILES                 <dbl> 2391.740, 171.390, 1588.060, 141.540, 18.000, 1…
$ AGE_UNKNOWN_MILES     <dbl> 371.800, 0.000, 0.000, 39.230, 0.000, 0.000, 10…
$ MILES_PRE_1940        <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ MILES_1940            <dbl> 10.000, 0.000, 151.840, 6.980, 0.000, 0.000, 13…
$ MILES_1950            <dbl> 276.300, 0.000, 16.940, 3.140, 1.000, 0.000, 16…
$ MILES_1960            <dbl> 183.680, 11.000, 206.420, 2.810, 3.000, 0.000, …
$ MILES_1970            <dbl> 69.000, 6.000, 0.000, 11.360, 2.000, 22.000, 64…
$ MILES_1980            <dbl> 23.000, 5.000, 41.000, 0.550, 

In [33]:
sample$AVG_AGE <- ((sample$MILES_PRE_1940 * 90 +
                    sample$MILES_1940 * 75 +
                    sample$MILES_1950 * 65 +
                    sample$MILES_1960 * 55 +
                    sample$MILES_1970 * 45 +
                    sample$MILES_1980 * 35 +
                    sample$MILES_1990 * 25 +
                    sample$MILES_2000 * 15 +
                    sample$MILES_2010 * 05) /
                   (sample$MILES_PRE_1940 +
                    sample$MILES_1940 +
                    sample$MILES_1950 +
                    sample$MILES_1960 +
                    sample$MILES_1970 +
                    sample$MILES_1980 +
                    sample$MILES_1990 +
                    sample$MILES_2000 +
                    sample$MILES_2010))

glimpse(sample)

Observations: 1,425
Variables: 23
Groups: OPERATOR_ID, COMMODITY [166]
$ YEAR                  <dbl> 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008,…
$ COMMODITY             <chr> "crude", "crude", "hvl", "non-hvl", "hvl", "non…
$ OPERATOR_ID           <chr> "SemGroup (Group)", "BP (Group)", "BP (Group)",…
$ NAME                  <chr> "SemGroup (Group)", "BP (Group)", "BP (Group)",…
$ MILES                 <dbl> 119.000, 24.000, 359.000, 762.000, 881.000, 341…
$ AGE_UNKNOWN_MILES     <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ MILES_PRE_1940        <dbl> 22.000, 0.000, 0.000, 152.000, 0.000, 0.000, 0.…
$ MILES_1940            <dbl> 22.000, 0.000, 0.000, 2.000, 0.000, 763.000, 0.…
$ MILES_1950            <dbl> 146.00, 4.00, 0.00, 166.00, 0.70, 668.20, 368.2…
$ MILES_1960            <dbl> 10.000, 7.000, 186.000, 80.000, 38.200, 581.000…
$ MILES_1970            <dbl> 6.000, 5.000, 168.000, 700.000, 581.800, 0.000,…
$ MILES_1980            <dbl> 2.000, 3.000, 33.000, 29.000, 

In [34]:
test <- select(sample, COMMODITY, YEAR, OPERATOR_ID)
test <- test[duplicated(test), ]
testit::assert(nrow(test) == 0)

## 9.6 Save

In [35]:
testit::assert(nrow(subset(sample, YEAR == 2008)) > 0)
feather::write_feather(sample, paste0("../preprocessed_data/sample_consolidated_", Sys.Date(), ".feather"))
haven::write_dta(sample, paste0("../stata_data/sample_consolidated_", Sys.Date(), ".dta"))